In [137]:
# Dependencies
import pandas as pd
import geopandas as gpd

In [138]:
yr = 2013

In [139]:
gdf = gpd.read_file(f'outputs/geojson/clean/ca_fire_{yr}.geojson')
gdf

,ALARM_DATE,CONT_DATE,CAUSE_ID,CAUSE,GIS_ACRES,AGENCY,FIRE_NAME,geometry
0,2013-08-03,2013-08-04,9,Miscellaneous,10.934740,CDF,VOLCANO,"POLYGON ((-120.78865 38.98208, -120.78863 38.9..."
1,2013-08-09,2013-08-09,9,Miscellaneous,206.080700,USF,BEE,"POLYGON ((-118.45753 34.56763, -118.45755 34.5..."
2,2013-06-09,2013-06-09,2,Equipment Use,17.784730,USF,DEL,"POLYGON ((-118.75073 34.64511, -118.75080 34.6..."
3,2013-09-07,2013-09-09,9,Miscellaneous,5.024302,USF,GULCH,"POLYGON ((-118.49904 34.55759, -118.49913 34.5..."
4,2013-05-13,2013-05-14,10,Vehicle,49.124420,USF,HIGHWAY,"POLYGON ((-118.69682 34.58156, -118.69681 34.5..."
...,...,...,...,...,...,...,...,...
292,2013-08-23,2013-11-07,1,Lightning,681.378600,NPS,WINDY PEAK,"POLYGON ((-118.59876 37.01091, -118.59738 37.0..."
293,2013-09-11,2013-10-28,1,Lightning,0.096645,NPS,WILLOW CREEK,"POLYGON ((-118.46359 36.43484, -118.46359 36.4..."
294,2013-07-23,2013-11-07,1,Lightning,48.824480,NPS,HOCKETT FIRE,"MULTIPOLYGON (((-118.66512 36.37504, -118.6650..."
295,2013-05-20,2013-06-06,11,Powerline,15.239750,NPS,SOTTY'S,"MULTIPOLYGON (((-117.32821 37.03159, -117.3281..."


In [141]:
test_df = pd.read_csv(f'outputs/prcp/{yr}/CA_{yr}_coords.csv')
stn_gdf = gpd.GeoDataFrame(test_df, geometry=gpd.points_from_xy(test_df.lon, test_df.lat), crs=4326)
stn_gdf

,station,lat,lon,elev,name,geometry
0,US1CAAL0001,37.8123,-122.2160,113.4,CA PIEDMONT 1.0 SE,POINT (-122.21600 37.81230)
1,US1CAAL0003,37.7169,-122.0585,163.4,CA CASTRO VALLEY 0.4 NNE,POINT (-122.05850 37.71690)
2,US1CAAL0004,37.6483,-121.8745,107.0,CA PLEASANTON 1.8 SSE,POINT (-121.87450 37.64830)
3,US1CAAL0006,37.8319,-122.2503,58.2,CA OAKLAND 4.4 NNW,POINT (-122.25030 37.83190)
4,US1CAAL0007,37.5150,-121.9523,9.1,CA FREMONT 2.6 ESE,POINT (-121.95230 37.51500)
...,...,...,...,...,...,...
1043,USW00093241,38.3775,-121.9586,31.7,CA VACAVILLE/NUT TREE AP ASOS,POINT (-121.95860 38.37750)
1044,USW00093242,36.9850,-120.1106,76.5,CA MADERA MUNI AP,POINT (-120.11060 36.98500)
1045,USW00093243,37.2381,-120.8825,23.8,CA MERCED 23 WSW CRN,POINT (-120.88250 37.23810)
1046,USW00093245,38.3208,-123.0747,19.2,CA BODEGA 6 WSW CRN,POINT (-123.07470 38.32080)


In [136]:
for y in [yr,yr-1]:
    df = pd.read_csv(f'outputs/prcp/{y}/CA_{y}_coords.csv')
    stn_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs=4326)
    gdf = gdf.to_crs(32610).sjoin_nearest(stn_gdf.to_crs(32610)).to_crs(4326)
    gdf = gdf.drop(columns=['index_right','elev','name','lat','lon']).rename(columns={'station':f'STN{y}'})
    gdf[f'STN{y}'].to_csv(f'outputs/fire/{yr}/stns_{y}.csv',index=False)
    print(gdf.columns)
    df = pd.read_csv(f'outputs/fire/{yr}/stns_{y}.csv').rename(columns={f'STN{y}':'station'})
    prcp_df = pd.read_csv(f'outputs/prcp/{y}/CA_{y}_prcp_data.csv')
    df = df.merge(prcp_df, on="station").set_index('station').drop(columns=['lat','lon','elev','name']).T
    df.index = df.index.str.slice(0,7)
    df = df.groupby(level=0).sum().round(2).T
    df.to_csv(f'outputs/fire/{yr}/prcp_{y}.csv')
    print(df.columns)

Index(['ALARM_DATE', 'CONT_DATE', 'CAUSE_ID', 'CAUSE', 'GIS_ACRES', 'AGENCY',
       'FIRE_NAME', 'geometry', 'STN2013'],
      dtype='object')
Index(['2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06',
       '2013-07', '2013-08', '2013-09', '2013-10', '2013-11', '2013-12'],
      dtype='object')


FileNotFoundError: [Errno 2] No such file or directory: 'outputs/prcp/2012/CA_2012_coords.csv'